![alt text](https://zewailcity.edu.eg/main/images/logo3.png)
#### Ezzat Eisawy 201901195
##### NN implementation projet

In [36]:
import numpy as np;
import pandas as pd
from numpy import exp
from numpy import array;
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

#### Building the model

In [37]:
class nn_classifier :
    def sig_activation(self,x) : return 1/(1+exp(-x));

    def relu_activation(self ,x) : return np.maximum(0,x);
        
    def __init__(self , hidden_layers = (5,5) , epochs = 20, learning_rate = 0.1 , activation_type = 'sigmoid') -> None:
        self.hidden_layers = hidden_layers;
        self.learning_rate = learning_rate ;
        self.activation_type = activation_type
        if activation_type == 'sigmoid' :
            self.activation = np.vectorize(self.sig_activation)  ;
        if activation_type == 'relu' :
             self.activation = np.vectorize(self.relu_activation)  ;
        self.epochs = epochs;

    def _init_layers(self, n_input,hidden_layers ,n_output ) :
        ''' Initialize all the weights with random nambers (-0.5 --> 0.5)
        '''
        total = len(hidden_layers) ;
        weights_list = np.empty(shape=(total+1) , dtype= object) ; 
        weights_list[0] = np.random.uniform(-0.5 , high = 0.5 , size  = (hidden_layers[0] , n_input))  # Inner layer

        for i in range(1,total) :   # Hidden layers
            weights_list[i] = np.random.uniform(low=- 0.5 , high = 0.5 , size  = (hidden_layers[i] , hidden_layers[i-1])) 

        weights_list[total] = np.random.uniform(low=-0.5 , high = 0.5, size  = ( n_output, hidden_layers[total-1]));  # output layers
        self.weights_list = weights_list;
        


    def _forward(self,x ) :
        ''' Forward propagation , returns the result value on each neuron on each layer 
        '''
        res = np.empty(shape= len(self.weights_list) , dtype= object );
        res[0] = self.activation(self.weights_list[0] @ np.array(x)) ;


        for i in range(1,len(self.weights_list)) :
            res[i] = self.activation (self.weights_list[i] @ res[i-1]) ;

        return res;

    def _backward(self,expected , outputs ,x) :
        ''' Backward propagation , Recalculates the weights 
        '''
        

        # Output layer
        deltas = np.empty(shape= len(self.weights_list) , dtype= object );
        last = len(self.weights_list) -1;

        diff = outputs[last] * (1 - outputs[last]) if self.activation_type =='sigmoid' else (outputs[last]>0 ).astype(np.int32)
        deltas[last]  = diff* (expected - outputs[last]) ; 

        self.weights_list[last] += self.learning_rate *np.outer(deltas[last] ,outputs[last-1] )    ; 

        
        for i in range(last-1, 0 , -1) :   # Inner layers
            diff = outputs[i] * (1 - outputs[i]) if self.activation_type =='sigmoid' else  (outputs[i]>0 ).astype(np.int32)
            deltas[i] = diff * np.sum(deltas[i+1] * self.weights_list[i+1].T , axis=1)
            self.weights_list[i] += self.learning_rate * np.outer( deltas[i] , outputs[i-1])  ; 
        

        #Input layer

        diff = outputs[0] * (1 - outputs[0]) if self.activation_type =='sigmoid' else (outputs[0]>0 ).astype(np.int32)
        
        deltas [0] = diff * np.sum(deltas[1] * self.weights_list[1].T , axis=1)
        self.weights_list[0] +=self.learning_rate * np.outer( deltas[0] ,x )  ;
        

    def fit(self,X , Y  , verbose = 1) :
        # Shuffle the X 
        shuffler = np.random.permutation(len(X))
        x = X[shuffler]
        y = Y[shuffler]
        # Extract the classes of the output
        classes = np.array(list(set(y)) );
        self.classes = classes
        print(classes)

        # Initialize weights
        self._init_layers(len(x[0]) , self.hidden_layers , len(classes ));

        # Epochs loop
        best_acc = 0
        for i in range (self.epochs) :
            mse_arr = np.zeros(len(x))
            itr = tqdm(range(len(x))) if verbose else range(len(x))
            for j in itr  :
                x_ = x[j]
                mapped = np.zeros(len(classes)) 
                mapped[classes == y[j]] = 1
                outputs = self._forward(x_);
                self._backward(mapped ,outputs ,x_)   # Update weights
                mse_arr[j] = np.mean(mapped - outputs[-1])**2
            acc = accuracy_score(y,self.predict(x))
            if acc > best_acc :
                best_acc = best_acc;
                self.best_weights = self.weights_list.copy()

            
            print(f'Epoch {i+1} #####' ,f'Mean square error = {( np.mean(mse_arr)) : .5f}' ,
            f', Accuracy = { acc :.3f}' )
        return self;

    def apply_best_accuracy(self) : 
        ''' Apply the weights with the best accuracy 
        '''
        self.weights_list = self.best_weights;
        return self

    def _predict(self,x) :    
        res = self._forward(x)[-1];
        idx =  np.argmax(res);

        return self.classes[idx];

    def predict(self , x) :    # Predict X 
        if hasattr(x, "__len__") : #return (np.vectorize(self._predict))(x) ; 
            if isinstance(self.classes[0] , str) :
                res = np.empty(len(x) ,dtype= object) 
            else :
                res = np.zeros(len(x) , dtype = np.int32 ) 
            for i in range(len(x)) :
                res[i] = self._predict(x[i])
            return res;
            
        return self._predict(x);


#### Testing with iris dataset

In [38]:
data_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/bezdekIris.data'
data = pd.read_csv(data_path , header= None)
data

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [39]:
input_data = data.iloc[:,0:4].copy()
from sklearn.preprocessing import StandardScaler
input_data = StandardScaler().fit_transform(input_data)
output_data = data.iloc[:,4].copy()

In [40]:
from sklearn.model_selection import train_test_split
input_train ,input_test , output_train , output_test = train_test_split(input_data , output_data , test_size=1/5, shuffle = 1)

##### Training the model

In [41]:
model = nn_classifier(hidden_layers=[100 ,50] ,learning_rate=1e-2 , epochs=50);
model.fit(array(input_train),array(output_train))

['Iris-versicolor' 'Iris-setosa' 'Iris-virginica']


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch 1 ##### Mean square error =  0.00307 , Accuracy = 0.358


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch 2 ##### Mean square error =  0.00103 , Accuracy = 0.583


...

Epoch 48 ##### Mean square error =  0.00165 , Accuracy = 0.892


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch 49 ##### Mean square error =  0.00166 , Accuracy = 0.892


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch 50 ##### Mean square error =  0.00167 , Accuracy = 0.892


##### Predicting test samples

In [42]:
print(f'Accuracy = {accuracy_score(output_test , model.predict(np.array(input_test))) : 0.3f}')

Accuracy =  0.633


#### Testing the model with digits dataset

In [43]:
from sklearn import datasets
digits = datasets.load_digits() 
x_data = digits.images.reshape(len(digits.images),8*8)
y_data = digits.target.copy()

In [44]:
x_train , x_test ,y_train,y_test = train_test_split(x_data,y_data,test_size = 1/4 , shuffle = 1)

##### Testing a model with two hidden layers ( 200 , 100)

In [45]:
model = nn_classifier(hidden_layers= [200 , 100] , epochs= 20 ,learning_rate= 1e-2)
model.fit(x_train , y_train)

[0 1 2 3 4 5 6 7 8 9]


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 1 ##### Mean square error =  0.01051 , Accuracy = 0.566


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 2 ##### Mean square error =  0.00118 , Accuracy = 0.695


...

Epoch 18 ##### Mean square error =  0.00042 , Accuracy = 0.988


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 19 ##### Mean square error =  0.00040 , Accuracy = 0.991


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 20 ##### Mean square error =  0.00037 , Accuracy = 0.991


In [46]:
print(f'Test Accuracy = {accuracy_score(y_test , model.predict(np.array(x_test))) : 0.3f}')

Test Accuracy =  0.960


##### Testing with one hidden layer (100)

In [47]:
model = nn_classifier(hidden_layers= [100] , epochs= 20 ,learning_rate= 1e-2)
model.fit(x_train , y_train)

[0 1 2 3 4 5 6 7 8 9]


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 1 ##### Mean square error =  0.00640 , Accuracy = 0.668


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 2 ##### Mean square error =  0.00171 , Accuracy = 0.795


...

Epoch 18 ##### Mean square error =  0.00041 , Accuracy = 0.976


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 19 ##### Mean square error =  0.00039 , Accuracy = 0.978


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 20 ##### Mean square error =  0.00037 , Accuracy = 0.979


In [48]:
print(f'Accuracy = {accuracy_score(y_test , model.predict(np.array(x_test))) : 0.3f}')

Accuracy =  0.956


##### Model with three hidden layers (50,40,20)

In [49]:
model = nn_classifier(hidden_layers= [50 , 40 , 20] , epochs= 100 ,learning_rate= 1e-2)
model.fit(x_train , y_train)

[0 1 2 3 4 5 6 7 8 9]


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 1 ##### Mean square error =  0.01713 , Accuracy = 0.125


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 2 ##### Mean square error =  0.00015 , Accuracy = 0.159


...

Epoch 98 ##### Mean square error =  0.00023 , Accuracy = 0.990


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 99 ##### Mean square error =  0.00023 , Accuracy = 0.990


  0%|          | 0/1347 [00:00<?, ?it/s]

Epoch 100 ##### Mean square error =  0.00022 , Accuracy = 0.990


In [50]:
print(f'Accuracy = {accuracy_score(y_test , model.predict(np.array(x_test))) : 0.3f}')

Accuracy =  0.949
